# developing and testing pynetstim

In [1]:
import nibabel as nib
import numpy as np

from surfer import Brain
from mayavi import mlab
%gui qt
from coordinates import Coords,Surf, FreesurferSurf, FreesurferCoords
from targets import Targets
from project import StimProject
import matplotlib.pyplot as plt
%matplotlib inline
#import seaborn as sns
#sns.set_context('notebook')
#sns.set_style('ticks')
from pymisc.plotting import clean_plot
from collections import defaultdict
from plotting import plotting_points
from tms_simulation import Simnibs

plotting.py:9: DeprecationWarning: Python2 support is deprecated and will be removed in a future release. Consider switching to Python3.
  from nilearn.plotting import plot_anat


## Visualization

In [2]:
subject = 'Broad_60_AV'
freesurfer_dir = '/Users/stadayon/Dropbox/projects/broad/data/freesurfer/subjects'
simnibs_dir = '/Users/stadayon/Dropbox/projects/broad/data/mri2mesh/{subject}'.format(subject=subject)
simnibs_simulation_dir = '/Users/stadayon/Dropbox/projects/broad/data/simnibs_simulations/{subject}'.format(subject=subject)
project_dir = '/Users/stadayon/Desktop/pynetstim_test/'
brainsight_file = '/Users/stadayon/Dropbox/projects/broad/data/sessions_data_raw/{subject}_sessions_data.txt'.format(subject=subject)

project = StimProject(subject,freesurfer_dir = freesurfer_dir,project_dir= project_dir,brainsight_file = brainsight_file, overwrite='all')
project.summary(plot_pulses=False)

Target
Sample
Planned
Session
Session
head model exists, if you want to recreate the head models, remove the directory:
/Users/stadayon/Dropbox/projects/broad/data/freesurfer/subjects/Broad_60_AV/bem


In [3]:
targets = project.brainsight_targets
samples = project.brainsight_samples

In [4]:
plotting_points(targets,'both','pial','pial',show_roi=True,show_name=True,scale_factor=.5)

[ simnibs ]WARNING: /Users/stadayon/anaconda2/envs/pynetstim/lib/python2.7/site-packages/mayavi/tools/helper_functions.py:107: DeprecationWarning: use "HasTraits.trait_get" instead
  traits = self.get(self.class_trait_names())



In [5]:
s = Simnibs(subject,simnibs_dir,simnibs_simulation_dir)

head model exists, if you want to recreate the head models, remove the directory:
/Users/stadayon/Dropbox/projects/broad/data/mri2mesh/Broad_60_AV/fs_Broad_60_AV/bem


In [6]:
targets.name

array(['R DLPFC', 'R OCCIPITAL', 'L DLPFC', 'L ANGGY', 'V1_BI_1',
       'V1_mono_1', 'V1_mono_2', 'V1_mono_3', 'V2_BI_1', 'V2_BI_2',
       'V2_MONO_1', 'DAN_Conf', 'DMN_Conf', 'DMN_Indiv', 'DAN_Indiv',
       'V3_BI_1', 'V3_BI_2', 'V3_MONO_1', 'V3_MONO_2', 'V3_MONO_3',
       'V4_BI_1', 'V4_MONO_1', 'V4_MONO_2', 'V4_MONO_3', 'V5_BI_1',
       'V5_BI_2', 'V5_MONO_1', 'V5_MONO_2', 'V6_BI_1', 'V6_BI_2',
       'V6_BI_3', 'V6_MONO_1', 'V6_MONO_2', 'V6_MONO_3'], dtype=object)

In [7]:
targets_inds = []
targets_inds.append(np.where(targets.name=='DMN_Conf')[0][0])
#targets_inds.append(np.where(targets.name=='LDLPFC')[0][0])

In [8]:
targets = Targets(targets.coordinates['ras_coord'][targets_inds,:], subject, freesurfer_dir , direction=targets.direction[targets_inds,:],
                  name =np.array(['DMN']))


In [9]:
s.add_targets(targets,'targets_DMN.csv',t12simnibs=True,project2skin=True)

190109-16:45:24,900 nipype.workflow INFO:
	 Workflow T1_to_simnibs_registration settings: ['check', 'execution', 'logging', 'monitoring']
190109-16:45:24,911 nipype.workflow INFO:
	 Running serially.
190109-16:45:24,915 nipype.workflow INFO:
	 [Node] Setting-up "T1_to_simnibs_registration.flirt" in "/Users/stadayon/Dropbox/projects/broad/data/simnibs_simulations/Broad_60_AV/T1_to_simnibs_registration/flirt".
190109-16:45:24,944 nipype.workflow INFO:
	 [Node] Running "flirt" ("nipype.interfaces.fsl.preprocess.FLIRT"), a CommandLine Interface with command:
flirt -in /Users/stadayon/Dropbox/projects/broad/data/mri2mesh/Broad_60_AV/m2m_Broad_60_AV/T1fs.nii.gz -ref /Users/stadayon/Dropbox/projects/broad/data/mri2mesh/Broad_60_AV/m2m_Broad_60_AV/T1fs_conform.nii.gz -out T1_in_Simnibs.nii.gz -omat T12Simnibs.mat -searchrx -180 180 -searchry -180 180 -searchrz -180 180
190109-16:48:30,127 nipype.interface INFO:
	 (4, 'Interrupted system call')


[ simnibs ]WARNING: /Users/stadayon/anaconda2/envs/pynetstim/lib/python2.7/site-packages/nipype/pipeline/engine/utils.py:307: DeprecationWarning: use "HasTraits.trait_set" instead
  result.outputs.set(**modify_paths(tosave, relative=True, basedir=cwd))



190109-16:48:30,777 nipype.workflow INFO:
	 [Node] Finished "T1_to_simnibs_registration.flirt".
190109-16:48:31,470 nipype.interface INFO:
	 stdout 2019-01-09T16:48:31.469037:Coordinates in Destination volume (in mm)
190109-16:48:31,479 nipype.interface INFO:
	 stdout 2019-01-09T16:48:31.479246:50.0223  -59.4653  11.5234
190109-16:48:31,483 nipype.interface INFO:
	 stdout 2019-01-09T16:48:31.479246:50.0223  -59.4653  11.5234


In [11]:
#s.run_simnibs('targets_DMN',didt=120e6)

In [54]:
t = 0
brain = Brain(subject_id='fs_'+subject,subjects_dir=simnibs_dir,surf='pial',hemi='both',offset=False)
sim_data = nib.freesurfer.read_morph_data('/Users/stadayon/Dropbox/projects/broad/data/simnibs_simulations/{subject}/simnibs_simulations/targets_DMN/subject_overlays/rh.{subject}_TMS_1-000{t}_MagVenture_MC_B70_scalar.central.E.norm'.format(subject=subject,t=t+1))
sim_data = sim_data/np.max(sim_data)

In [55]:
brain.add_overlay(sim_data,hemi='rh',min=0.75,max=.95)
brain.add_foci(s.targets.coordinates['ras_tkr_coord'][t,:],hemi=s.targets.hemi[t])
mlab.show()